# LAB 01: MỐI QUAN HỆ CỦA DỮ LIỆU

## 1. Thông tin nhóm

| MSSV     | HỌ VÀ TÊN          |
|:--------:|:-------------------|
| 20127323 | Võ Nhật Tân        |
| 20127447 | Ngô Đức Bảo        |
| 20127275 | Lê Nguyễn Nhật Phú |
| 20127681 | Nguyễn Thiên Phúc  |


## 2. Lấy thông tin từ website

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
from bs4 import BeautifulSoup

np.set_printoptions(formatter=dict(float='{:0.2f}'.format))

## Trực quan hóa dữ liệu

## Sử dụng dữ liệu của khu vực Châu Á trong vòng 7 ngày (từ 26/2/2023 đến 4/3/2023)

In [5]:
df1 = pd.read_csv('./data/26-02-2023.csv')
df2 = pd.read_csv('./data/27-02-2023.csv')
df3 = pd.read_csv('./data/28-02-2023.csv')
df4 = pd.read_csv('./data/01-03-2023.csv')
df5 = pd.read_csv('./data/02-03-2023.csv')
df6 = pd.read_csv('./data/03-03-2023.csv')
df7 = pd.read_csv('./data/04-03-2023.csv')

## Tiền xử lý

In [6]:
df = df1[df1['Country,Other'] == 'Vietnam'].copy()
df['Date'] = '26-02-2023'
df = df.append(df2[df2['Continent'] == 'Asia'])
df.loc[df['Date'].isna(), 'Date'] = '27-02-2023'
df = df.append(df3[df3['Continent'] == 'Asia'])
df.loc[df['Date'].isna(), 'Date'] = '28-02-2023'
df = df.append(df4[df4['Continent'] == 'Asia'])
df.loc[df['Date'].isna(), 'Date'] = '01-03-2023'
df = df.append(df5[df5['Continent'] == 'Asia'])
df.loc[df['Date'].isna(), 'Date'] = '02-03-2023'
df = df.append(df6[df6['Continent'] == 'Asia'])
df.loc[df['Date'].isna(), 'Date'] = '03-03-2023'
df = df.append(df7[df7['Continent'] == 'Asia'])
df.loc[df['Date'].isna(), 'Date'] = '04-03-2023'

df.reset_index(drop=True, inplace=True)

df.head(10)

C:\Users\KuunVo\AppData\Local\Temp\ipykernel_10924\2925247170.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2[df2['Continent'] == 'Asia'])
C:\Users\KuunVo\AppData\Local\Temp\ipykernel_10924\2925247170.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df3[df3['Continent'] == 'Asia'])
C:\Users\KuunVo\AppData\Local\Temp\ipykernel_10924\2925247170.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df4[df4['Continent'] == 'Asia'])
C:\Users\KuunVo\AppData\Local\Temp\ipykernel_10924\2925247170.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df5[df5['Continent'] == 'Asia

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Population,Continent,Date
0,Vietnam,11526901,10.0,43186.0,0.0,10614760.0,1.0,868955.0,116488.0,436.0,85826548.0,867342.0,9.895354e+07,Asia,26-02-2023
1,India,44686371,169.0,530771.0,1.0,44153343.0,140.0,2257.0,31768.0,377.0,918535118.0,653003.0,1.406632e+09,Asia,27-02-2023
2,Japan,33190564,5598.0,72320.0,52.0,21683904.0,1535.0,11434340.0,264288.0,576.0,95014492.0,756576.0,1.255848e+08,Asia,27-02-2023
3,S. Korea,30502904,4026.0,33961.0,15.0,30278033.0,0.0,190910.0,594252.0,662.0,15804065.0,307892.0,5.132990e+07,Asia,27-02-2023
4,Turkey,17042722,0.0,101492.0,0.0,NaN,NaN,NaN,199186.0,1186.0,162743369.0,1902052.0,8.556198e+07,Asia,27-02-2023
5,Vietnam,11526905,4.0,43186.0,0.0,10614763.0,3.0,868956.0,116488.0,436.0,85826548.0,867342.0,9.895354e+07,Asia,27-02-2023
6,Taiwan,10033107,7744.0,17908.0,44.0,9690990.0,23339.0,324209.0,419996.0,750.0,30742304.0,1286903.0,2.388860e+07,Asia,27-02-2023
7,Iran,7567824,122.0,144845.0,3.0,7339283.0,78.0,83696.0,87975.0,1684.0,55034937.0,639771.0,8.602284e+07,Asia,27-02-2023
8,Indonesia,6735780,152.0,160911.0,3.0,6571457.0,270.0,3412.0,24131.0,576.0,114158919.0,408975.0,2.791345e+08,Asia,27-02-2023
9,Malaysia,5042585,190.0,36957.0,0.0,4996409.0,177.0,9219.0,151972.0,1114.0,68079467.0,2051756.0,3.318107e+07,Asia,27-02-2023


In [7]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
df.dtypes

Country,Other               object
TotalCases                   int64
NewCases                   float64
TotalDeaths                float64
NewDeaths                  float64
TotalRecovered             float64
NewRecovered               float64
ActiveCases                float64
Tot Cases/1M pop           float64
Deaths/1M pop              float64
TotalTests                 float64
Tests/ 1M pop              float64
Population                 float64
Continent                   object
Date                datetime64[ns]
dtype: object

In [8]:
df.isna().sum()
df.dropna(inplace=True)

In [9]:
df = df.groupby('Country,Other').apply(lambda x: x.sort_values('Date', ascending=True)).reset_index(drop=True)
df

C:\Users\KuunVo\AppData\Local\Temp\ipykernel_10924\1785859924.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('Country,Other').apply(lambda x: x.sort_values('Date', ascending=True)).reset_index(drop=True)


,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Population,Continent,Date
0,Vietnam,11526901,10.0,43186.0,0.0,10614760.0,1.0,868955.0,116488.0,436.0,85826548.0,867342.0,9.895354e+07,Asia,2023-02-26
1,India,44686371,169.0,530771.0,1.0,44153343.0,140.0,2257.0,31768.0,377.0,918535118.0,653003.0,1.406632e+09,Asia,2023-02-27
2,Japan,33190564,5598.0,72320.0,52.0,21683904.0,1535.0,11434340.0,264288.0,576.0,95014492.0,756576.0,1.255848e+08,Asia,2023-02-27
3,S. Korea,30502904,4026.0,33961.0,15.0,30278033.0,0.0,190910.0,594252.0,662.0,15804065.0,307892.0,5.132990e+07,Asia,2023-02-27
4,Vietnam,11526905,4.0,43186.0,0.0,10614763.0,3.0,868956.0,116488.0,436.0,85826548.0,867342.0,9.895354e+07,Asia,2023-02-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,Syria,57467,0.0,3164.0,0.0,54303.0,0.0,0.0,2968.0,163.0,146269.0,7553.0,1.936481e+07,Asia,2023-03-04
285,Timor-Leste,23418,0.0,138.0,0.0,23102.0,0.0,178.0,17101.0,101.0,278529.0,203391.0,1.369429e+06,Asia,2023-03-04
286,Tajikistan,17786,0.0,125.0,0.0,17264.0,0.0,397.0,1786.0,13.0,0.0,0.0,9.957464e+06,Asia,2023-03-04
287,Yemen,11945,0.0,2159.0,0.0,9124.0,0.0,662.0,383.0,69.0,329592.0,10579.0,3.115487e+07,Asia,2023-03-04


In [10]:
numeric_cols = df.select_dtypes(exclude=[object, datetime]).columns
numeric_cols
for col in numeric_cols:
    print(f'Column {col} - Value counts in each bins: ')
    print(df[col].value_counts(bins=3, sort=False))
    print('-'*30)

Column TotalCases - Value counts in each bins: 
(-41170.30699999999, 14898282.667]    271
(14898282.667, 29793051.333]            0
(29793051.333, 44687820.0]             18
Name: TotalCases, dtype: int64
------------------------------
Column NewCases - Value counts in each bins: 
(-14.524999999999999, 4841.333]    272
(4841.333, 9682.667]                 3
(9682.667, 14524.0]                 14
Name: NewCases, dtype: int64
------------------------------
Column TotalDeaths - Value counts in each bins: 
(-509.755, 176939.0]    283
(176939.0, 353857.0]      0
(353857.0, 530775.0]      6
Name: TotalDeaths, dtype: int64
------------------------------
Column NewDeaths - Value counts in each bins: 
(-0.1, 33.0]    278
(33.0, 66.0]      5
(66.0, 99.0]      6
Name: NewDeaths, dtype: int64
------------------------------
Column TotalRecovered - Value counts in each bins: 
(-40763.867999999995, 14720342.667]    271
(14720342.667, 29437298.333]             6
(29437298.333, 44154254.0]             

- Ta trực quan mối tương quan giữa các biến bằng ma trận correlation

In [11]:
corr = df[numeric_cols].corr()
corr.style.background_gradient(cmap='coolwarm')

C:\Users\KuunVo\AppData\Local\Temp\ipykernel_10924\3922675089.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df[numeric_cols].corr()


,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Population
TotalCases,1.000000,0.574484,0.746545,0.512188,0.984965,0.321333,0.500641,0.184953,-0.019027,0.702822,-0.107798,0.471081
NewCases,0.574484,1.000000,0.052784,0.840189,0.509235,0.758437,0.584358,0.382499,0.000381,0.018314,-0.066436,-0.019711
TotalDeaths,0.746545,0.052784,1.000000,0.083790,0.790774,-0.002945,0.079454,-0.138272,0.000985,0.918751,-0.116697,0.659617
NewDeaths,0.512188,0.840189,0.083790,1.000000,0.394687,0.465889,0.810949,0.251028,0.016012,0.038469,-0.050727,-0.008387
TotalRecovered,0.984965,0.509235,0.790774,0.394687,1.000000,0.339235,0.343651,0.183486,-0.016263,0.748321,-0.105855,0.504899
NewRecovered,0.321333,0.758437,-0.002945,0.465889,0.339235,1.000000,0.060926,0.369853,0.024161,-0.015365,-0.021375,-0.039682
ActiveCases,0.500641,0.584358,0.079454,0.810949,0.343651,0.060926,1.000000,0.099615,-0.022454,0.056925,-0.053786,0.019687
Tot Cases/1M pop,0.184953,0.382499,-0.138272,0.251028,0.183486,0.369853,0.099615,1.000000,0.414272,-0.110529,0.356399,-0.218222
Deaths/1M pop,-0.019027,0.000381,0.000985,0.016012,-0.016263,0.024161,-0.022454,0.414272,1.000000,-0.066295,0.190729,-0.169002
TotalTests,0.702822,0.018314,0.918751,0.038469,0.748321,-0.015365,0.056925,-0.110529,-0.066295,1.000000,0.108220,0.764746


Từ ma trận correlation trên, ta thấy rằng các cột TotalCases, TotalDeaths, TotalRecovered, ActiveCases, Serious,Critical, Tot Cases/1M pop, Deaths/1M pop, TotalTests, Tests/1M pop có mối tương quan cao với nhau


In [12]:
top_5_Tot_Cases_1M_pop_df = df.sort_values(by=['TotalCases'], ascending=False).iloc[1:26]
top_5_Tot_Cases_1M_pop = top_5_Tot_Cases_1M_pop_df['Country,Other'].unique()
top_5_Tot_Cases_1M_pop

array(['India', 'Japan', 'S. Korea', 'Vietnam', 'Taiwan'], dtype=object)

Ta thấy, 5 quốc gia có độ lây lan cao nhất khu vực châu Á là India, Japan, S.Korea, Turkey và Việt Nam
Tốc độ lây lan ở India là cao nhất do dân số đông nhất khu vực Châu Á

In [13]:
#compare the total deaths in the top 5 countries with line chart
fig = px.line(df[df['Country,Other'].isin(top_5_Tot_Cases_1M_pop)], x='Date', y='NewRecovered', color='Country,Other')
fig.show()

In [14]:
fig = px.line(df[df['Country,Other'].isin(top_5_Tot_Cases_1M_pop)], x='Date', y='NewDeaths', color='Country,Other')
fig.show()

Trong vòng 7 ngày, S.Korea là quốc gia có số ca hồi phục nhiều nhất và số ca tử vong giảm dần, có thể nói rằng S.Korea đang đứng đầu trong việc kiểm soát dịch bệnh

In [15]:
asian_countries = ['China', 'India', 'Indonesia', 'Pakistan', 'Bangladesh', 'Japan', 'Brunei',
                   'Philippines', 'Vietnam', 'Iran', 'Turkey', 'Thailand', 'Myanmar', 'S. Korea', 'DPRK',
                   'Iraq', 'Afghanistan', 'Saudi Arabia', 'Uzbekistan', 'Malaysia', 'Nepal', 'Yemen', 
                   'Taiwan', 'Sri Lanka', 'Kazakhstan', 'Syria', 'Cambodia', 'Jordan', 
                   'Azerbaijan', 'UAE', 'Tajikistan', 'Israel', 'Hong Kong', 'Laos', 
                   'Lebanon', 'Kyrgyzstan', 'Singapore', 'Palestine', 'Oman', 'Kuwait', 
                   'Georgia', 'Mongolia', 'Armenia', 'Qatar', 'Bahrain', 'Timor-Leste', 'Cyprus', 'Bhutan', 'Macao', 'Maldives']

df_asia = df[df['Country,Other'].isin(asian_countries)]
# Tạo biểu đồ scatter plot
fig = px.scatter(df_asia, x='TotalCases', y='NewCases', color='Country,Other', log_x=True, log_y=True, size_max=50, title='Total Cases vs. New Cases in Asia countries')
fig.show()

Từ biểu đồ trên, mối tương quan giữa biến NewCases với biến TotalCases là tuyến tính, với số ca nhiễm mới cao nhất là ở các nước: Taiwan, S.Korea và Japan


In [16]:
#total test in asia countries with bar chart
fig = px.bar(df_asia, x='Country,Other', y='TotalTests', color='Country,Other', title='TotalTests')
fig.show()

Về việc thực hiện test covid-19, ta thấy rằng các quốc gia có số ca nhiễm cao nhất thì số ca test cũng cao nhất, đồng thời số ca test cũng cao nhất ở các quốc gia có dân số đông nhất khu vực Châu Á